<a href="https://colab.research.google.com/github/itforankit/GenerativeAI/blob/main/GANs_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

GAN -> Generative Adversarial Network

GAN -> Generate new data set based on the traiing data set and new dataset look slike training dataset on which the model ois trained


Human faces ----> generate faces which looks like human faces

GAN

1. Generator [unsupevised] : take random input (noise) and generate samples (image,text,audio) generate real looking sample which are indistinguishable form the training data.

  Input: take random input(noisa) -> latent dim 100 -> sample form the normal distribution

  Ouput: neuorns in output layer of generator ? 784 -> 28X28

  Generator learns by reviewing the gfeedback from the discriminator [good job -> loss fucntion is small] -> backward propogation to update the weights of generator such that the loss funsctions of the discruminator is maximized

2. Discriminator[superised]: distinguish between the real and fake data
trained on real and fake data [generated images generated by generator]

NN

Neuorns -> o/p of discriminator -> 1 Neuron

I/P of discriiminator  ->image(dim of the image) ex: 28X28=784




In [13]:
#define the generator model
def build_generator(latent_dim):
  generator = keras.Sequential([
      layers.Dense(128, input_dim=latent_dim, activation='relu'),
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28,28))
  ])
  return generator

#define the Discriminator model
def build_discriminator(input_shape):
  discriminator = keras.Sequential([
      layers.Flatten(input_shape=input_shape),
      layers.Dense(128, activation='relu'),
      #layers.Droupout(0.5),
      layers.Dense(1, activation='sigmoid')
  ])
  return discriminator

#combine Generator and Discriminator in to GAN model
def build_gan(generator, discriminator):
  discriminator.trainable =False
  gan=keras.Sequential([
      generator,
      discriminator
  ])
  return gan

In [14]:
(x_train, _), (_, _) = keras.datasets.mnist.load_data()
x_train = x_train / 255.0
#x_train = x_train.reshape((-1, 28, 28))

In [19]:
#set the parameters
latent_dim =100
input_shape =(28,28)
epochs =range(10000)
batch_size=128

#build and compile the models
generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape)
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

gan=build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

Do these steps iteratively

fake images --> generator(128 fake images) label -> 0 (fake images)

real images --> x_train(60,000 -> 128 real images) label -> 1 (real images)[128]

fake + real images -> 256 images (labelled data set (faked/real))

train our discriminator --> 256 images

sample 100 points from normal distribution  --> inout to generator  --> 256 iamges

mislabelled y --> set to 1 --> 256  ---> real images

freeze the weights disriminator ( the weight will not be learned)

train gan (gen + disc) --> supervised --> input random noise --> output (fake not fake) mislabelled 7=y

In [ ]:
import matplotlib.pyplot as plt
#training loop
for epoch in epochs:
  #smaple random noise for generator inout
  noise = np.random.normal(0,1,(batch_size, latent_dim))
  #generate fake images from noise
  generated_images = generator.predict(noise) #128 batch_size

  #sample real images
  real_images = x_train[np.random.randint(0,x_train.shape[0],batch_size)] # 128 real images

  #concatenate real and generated images for discriminator training
  x_combined = np.concatenate([real_images,generated_images])

  #labels for real(1) and generated images(0)
  y_combined = np.concatenate([np.ones((batch_size,1)), np.zeros((batch_size,1))])

  #add noise to labels for smoother training
  #y_combined += 0.05 * np.random.random(y_combined.shape)

  #train discriminator
  discriminator.trainable =True
  d_loss = discriminator.train_on_batch(x_combined, y_combined) # weights are updating

  #train generator
  noise = np.random.normal(0,1,(batch_size, latent_dim))
  y_mislabeled = np.ones((batch_size,1)) # internally making the fake images as real
  discriminator.trainable = False
  g_loss = gan.train_on_batch(noise, y_mislabeled)

  #print progress
  if epoch%100==0:
    print(f"Epoch: {epoch}, Discriminator loss: {d_loss[0]}, Disicriiminator Accuracy: {100 * d_loss[1]:.2f}%, Generstor loss: {g_loss}")
  #plot generated images at ertain intervals
  if epoch%1000==0:
    plt.figure(figsize=(10,10))
    for i in range(25):
      plt.subplot(5,5,i+1)
      plt.imshow(generated_images[i], cmap='gray')
      plt.axis('off')
    plt.tight_layout()
    plt.show()